In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go

# --- CONFIGURATION ---
ASSETS = {
    'EURUSD': 'EURUSD=X', 'USDCHF': 'USDCHF=X', 'GBPUSD': 'GBPUSD=X',
    'USDCAD': 'USDCAD=X', 'BTCUSD': 'BTC-USD', 'ETHUSD': 'ETH-USD',
    'XAUUSD': 'XAUUSD=X', 'XAGUSD': 'XAGUSD=X', 'SP500m': '^GSPC', 'UK100': '^FTSE'
}

TIMEFRAME_SETTINGS = {
    '1WK': ('1wk', '10y'), 'D1': ('1d', '5y'), '4H': ('4h', '2y'),
    '1H': ('1h', '2y'), 'M30': ('30m', '60d'), 'M15': ('15m', '60d')
}

# Strategy Tuning
LOOKBACK_PERIOD = 14
BODY_MULTIPLIER = 1.5
BOX_WIDTH = 30  # How many candles to trace the FVG forward


def flatten_yf_columns(df):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    return df


def detect_fvg_logic(data):
    """Detects FVGs and checks for Mitigation based on Close price."""
    fvg_list = [None] * len(data)
    time_col = 'Datetime' if 'Datetime' in data.columns else 'Date'

    for i in range(2, len(data)):
        # 1. CORE FVG DETECTION
        h1, l1 = data['High'].iloc[i-2], data['Low'].iloc[i-2]
        o2, c2 = data['Open'].iloc[i-1], data['Close'].iloc[i-1]
        h3, l3 = data['High'].iloc[i], data['Low'].iloc[i]

        mid_body = abs(c2 - o2)
        avg_body = abs(data['Close'] - data['Open']
                       ).iloc[max(0, i-1-LOOKBACK_PERIOD):i-1].mean()
        min_gap = mid_body * 0.10

        fvg_type, bottom, top = None, 0, 0
        if l3 > h1 and mid_body > (avg_body * BODY_MULTIPLIER) and (l3 - h1 > min_gap):
            fvg_type, bottom, top = 'bullish', h1, l3
        elif h3 < l1 and mid_body > (avg_body * BODY_MULTIPLIER) and (l1 - h3 > min_gap):
            fvg_type, bottom, top = 'bearish', l1, h3

        if fvg_type:
            is_mitigated = False
            # 2. CHECK FOR CLOSES INSIDE THE FVG
            check_range = min(i + BOX_WIDTH, len(data))
            for j in range(i + 1, check_range):
                close_j = data['Close'].iloc[j]

                # Mitigation Rule: Close price enters the FVG region
                if fvg_type == 'bullish' and close_j <= top:
                    is_mitigated = True
                    break
                elif fvg_type == 'bearish' and close_j >= bottom:
                    is_mitigated = True
                    break

            fvg_list[i] = {
                'type': fvg_type, 'bottom': bottom, 'top': top,
                'start': data[time_col].iloc[i-1], 'idx': i,
                'mitigated': is_mitigated
            }
    return fvg_list


def plot_fvg_signals(pair, timeframe, candle_limit=120):
    interval, period = TIMEFRAME_SETTINGS[timeframe]
    df = yf.download(ASSETS[pair], period=period,
                     interval=interval, auto_adjust=False, progress=False)
    if df.empty:
        return

    df = flatten_yf_columns(df).reset_index()
    time_col = 'Datetime' if 'Datetime' in df.columns else 'Date'

    df['FVG_RESULTS'] = detect_fvg_logic(df)
    df_plot = df.tail(candle_limit).copy()

    fig = go.Figure(data=[go.Candlestick(
        x=df_plot[time_col], open=df_plot["Open"], high=df_plot["High"],
        low=df_plot["Low"], close=df_plot["Close"], name=pair
    )])

    # Get the very latest close price for live mitigation check
    latest_close = df['Close'].iloc[-1]

    for _, row in df_plot.iterrows():
        res = row['FVG_RESULTS']
        if res:
            # INSTANT LIVE CHECK: Does current price close inside the box?
            live_hit = False
            if res['type'] == 'bullish' and latest_close <= res['top']:
                live_hit = True
            elif res['type'] == 'bearish' and latest_close >= res['bottom']:
                live_hit = True

            end_idx = min(res['idx'] + BOX_WIDTH, len(df) - 1)
            end_time = df[time_col].iloc[end_idx]

            # Use grey if historically mitigated OR currently hit by the latest candle
            if res['mitigated'] or live_hit:
                color = "rgba(180, 180, 180, 0.25)"
            else:
                color = "rgba(0, 255, 150, 0.4)" if res['type'] == 'bullish' else "rgba(255, 50, 50, 0.4)"

            fig.add_shape(type="rect", x0=res['start'], x1=end_time, y0=res['bottom'], y1=res['top'],
                          fillcolor=color, layer="below", line=dict(width=0))

    fig.update_layout(
        title=f"<b>{pair} {timeframe}</b> - FVG Mitigation (Close-Only)",
        template="plotly_dark", xaxis_rangeslider_visible=False,
        width=1200, height=750
    )
    fig.show()


if __name__ == "__main__":
    plot_fvg_signals('BTCUSD', '4H', candle_limit=100)